**Hey Luit, hier alvast een opzet van de opdracht. Ik zal per blokje kort even aangeven of het boeiend voor jou is.**

Dit is **niet** belangrijk voor jouw onderdelen, dit is alleen hier om het inladen van bestanden te fixen

In [72]:
def format_add(format_list, input_data, target_data):
    ### Constructs a dictionary using input_data and 
    ### the format_list as the dictionary format and 
    ### appends the dictionary to the target data.
    
    appender_dict = {}
    
    for count, item in enumerate(format_list):
        try:
            appender_dict.update({item: input_data[count]})
        except IndexError:
            appender_dict.update({item: None})
            
    target_data.append(appender_dict)

def build_file(format_list, file_path, delimiter):
    ### Reads a file and converts the delimiter separated data 
    ### in the file, based on the format provided by format_list, 
    ### into a dictionary list.
    
    new_list = []
    
    with open(file_path) as file:
        print(f"Unpacking: '{file_path}'")
    
        for line in file:
            stripped_line = line.strip().split(delimiter)
            format_add(format_list, stripped_line, new_list)
        
    print("Unpacked succesfully")
    
    return new_list

Hier staan de drie belangrijke datasets.

```user_data``` is een lijst van dictionaries waar elke dict een regel is uit de oorspronkelijke data bestanden. De keys zie je in de lijst van ```build_file()```. ```movie_data```en ```rating_data``` hebben het zelfde princiepe.

Als je ```user_data[0]``` gebruikt, krijg je dus een hele regel. Let op, UIDs en ```[id]``` zijn mogelijk niet 1 op 1! (zie ```find_on_key_and_value``` voor makkelijker zoeken naar gebruikers/movies/ratings/andere dingen).

In [73]:
file_path_users = "./ml-1m/users.dat"
file_path_movies = "./ml-1m/movies.dat"
file_path_ratings = "./ml-1m/ratings.dat"

user_data = build_file(
    ["uid", "gender", "age", "occupation", "zip-code"],
    file_path_users,
    "::"
)

movie_data = build_file(
    ["mid", "title", "genre"],
    file_path_movies,
    "::"
)

rating_data = build_file(
    ["uid", "mid", "rating", "timestamp"],
    file_path_ratings,
    "::"
)

Unpacking: './ml-1m/users.dat'
Unpacked succesfully
Unpacking: './ml-1m/movies.dat'
Unpacked succesfully
Unpacking: './ml-1m/ratings.dat'
Unpacked succesfully


Gebruik dit om een entry te zoeken gebaseerd op een key (zoals *uid*, *gender*, *title*) en de verwachtte waarde. **Let op** dit retourneerd een lijst (kunnen dus meerdere entries zijn), maar als je maar 1 ding zoekt doe je gewoon: ```find_on_key_and_value(blah, blah, blah)[0]``` of ```find_on_key_and_value(blah, blah, blah)[:1]``` (als er niets gevonden is krijg je een lege lijst)

In [94]:
def find_on_key_and_value(target_data, key, value):
    ### Filter the data based on a key-value combination in the list of dictionaries.
    return list(filter(lambda obj : obj[key] == value, target_data))

Hier een algemeen gemiddelde, niet zo spannend denk ik, maar misschien nodig bij de twee grotere methodes.

Hier moet je vooral niet kijken naar wat de code is, want comprehension is lekker niet zo leesbaar, maar dit zoekt alle gemiddelden voor een bepaalde ```target_mean``` gebaseerd op een ```target_key```.

Dus: ```bind_mean(rating_data, user_data, "uid", "rating", all_rating_mean)``` zoekt voor alle gebruikers (*uid*) de gemiddelde rating die die gebruiker heeft gegeven en maakt vervolgens een dictionary met daarin de gemiddelde van die gebruikers gekoppeld aan de uid van die gebruiker.

Oh ja, **let op**, de dataset is nogal groot, dus let er op dat dit even kan duren voordat het klaar is (5 minuten ofzo).

In [647]:
from numpy import mean, isnan

def bind_mean(bind_data_source, bind_data_target, target_key, target_mean, default=None):
    ### Creates a dictionary of target_keys with their respective means. 
    ### If the target key does not exist in the source_data, then a default value will be set.
    return {l_obj[target_key]: x if not isnan(x := mean([int(item[target_mean]) for item in filter(lambda _r : _r[target_key] == l_obj[target_key], bind_data_source)])) else default for l_obj in bind_data_target}

In [658]:
import numpy as np

def construct_M_array(row_size, column_size, data, make_nan=True):
    M_array = np.zeros((row_size, column_size))

    for item in data:
        itv = list(item.values())
        M_array[int(itv[0]) - 1, int(itv[1]) - 1] = float(itv[2])

    if make_nan:
        M_array = np.where(M_array == 0., np.NaN, M_array)
        
    return M_array

def get_row_col_mean(data):
    return np.nanmean(data, axis=1), np.nanmean(data, axis=0)

In [807]:
import numpy as np

def RSME_two_arrays(arr_1, arr_2):
    return np.sqrt(np.nanmean(((arr_1 - arr_2) ** 2)))

def MAE_two_arrays(arr_1, arr_2):
    return np.nanmean(np.absolute(arr_1 - arr_2))

In [794]:
import numpy as np
import warnings

def replace_nans(arr):
    copy_arr = np.copy(arr)
    
    inds = np.where(np.isnan(copy_arr))

    copy_arr[inds] = np.take(np.nanmean(copy_arr, axis=0), inds[1])
    
    return copy_arr

def preprocess_array(arr):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        
        ret_arr = np.copy(arr)
        
        row_mean = np.zeros((arr.shape[0], ))
        column_mean = np.zeros((arr.shape[1], ))
        
        for row in range(ret_arr.shape[0]):
            row_mean[row] = np.nanmean(ret_arr[row])
            ret_arr[row] = ret_arr[row] - row_mean[row]
        for column in range(ret_arr.shape[1]):
            column_mean[column] = np.nanmean(ret_arr[:,column])
            ret_arr[:,column] = ret_arr[:,column] - column_mean[column]
        return ret_arr, row_mean, column_mean
    
def unprocess_array(arr, row_mean, column_mean):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        
        ret_arr = np.copy(arr)

        for column in range(ret_arr.shape[1]):
            ret_arr[:,column] = ret_arr[:,column] + column_mean[column]
        for row in range(arr.shape[0]):
            ret_arr[row] = arr[row] + row_mean[row]
        return ret_arr

Split the data in een trainings- en testset


#Ik neem aan alleen de rating_data? dit zouden ze ook alle drie kunnen zijn maar logischerwijs heb je niks aan de rest van de data zonder rating dus dacht ik dat dit voldoende zou zijn. 

In [758]:
import numpy as np
from sklearn.model_selection import train_test_split

Rating_train, Rating_test = train_test_split(rating_data, test_size=0.2, random_state=42)

training_array = construct_M_array(int(user_data[-1]['uid']), int(movie_data[-1]['mid']), Rating_train)

testing_array = construct_M_array(int(user_data[-1]['uid']), int(movie_data[-1]['mid']), Rating_test)

Hier doen  we wat leuke testjes op de trainingsset

#Deze had jij al staan, ik heb enkel de rating_data naar rating_train gebracht omdat dit de enige data is waar nu naar gekeken mag worden en R_global is hetzelfde als all_rating_mean

In [675]:
from sklearn.linear_model import LinearRegression

#the global average rating 
R_global = np.nanmean(training_array)

#the average rating per user & item 
R_user, R_item = get_row_col_mean(training_array)

R_user = np.nan_to_num(R_user, nan=np.nanmean(R_user))

R_item = np.nan_to_num(R_item, nan=np.nanmean(R_item))

<ipython-input-658-c74362966e6a>:16: RuntimeWarning: Mean of empty slice
  return np.nanmean(data, axis=1), np.nanmean(data, axis=0)


In [676]:
for i, item in enumerate(R_item):
    if np.isnan(item):
        print(i)

# Naive approaches

#Hier is regressie nodig en dus de kritieke code. Om LR.fit uit te voeren hebben we arrays nodig met een X en Y met de volgende voorwaarde:
    - X: X{array-like, sparse matrix} of shape (n_samples, n_features), // Training data.
    - Y: array-like of shape (n_samples,) or (n_samples, n_targets), // Target values. Will be cast to X’s dtype if necessary.

Hierbij mijn vraag of LR.score gebruikt moet worden of LR.predict en wat precies de waardes zijn die in LR.fit moeten zitten. Dit is op dit moment in ieder geval nog **niet helemaal correct** !!

In [743]:
R_item.size

3952

In [711]:
from numpy import array

#an “optimal” linear combination of the two averages (per user and per item) without gamma
LR_1 = LinearRegression()
LR_2 = LinearRegression()

user_ids_array = np.arange(1, int(user_data[-1]['uid']) + 1, 1)

print 

LR_1.fit(user_ids_array.reshape(-1,1), R_user)
alpha = LR_1.coef_

item_ids_array = np.arange(1, int(movie_data[-1]['mid']) + 1, 1)

LR_2.fit(item_ids_array.reshape(-1,1), R_item)
beta = LR_2.coef_

gamma = (LR_1.intercept_ + LR_2.intercept_) / 2

Berekenen van de resultaten van de training, de RMSE!

#Dit klopt buiten het feit dat alpha en beta nog niet op orde zijn.

In [744]:
#Root mean squared error
RMSE_global = 0
RMSE_item = 0
RMSE_user = 0
RMSE_user_item = 0
RMSE_user_item_gamma = 0

for rating in Rating_test:
    RMSE_global += (int(rating['rating']) - int(R_global))**2    
    
    try:
        RMSE_item += (int(rating['rating']) - R_item[int(rating['mid']) - 1])**2
    except:
        RMSE_item += (int(rating['rating']) - int(R_global))**2
        
    try:
        RMSE_user += (int(rating['rating']) - R_user[int(rating['uid'])  - 1])**2
    except:
        RMSE_user += (int(rating['rating']) - int(R_global))**2
    
    try:
        RMSE_user_item += (int(rating['rating']) - int((alpha*R_user[int(rating['uid'])  - 1] + beta*R_item[int(rating['mid'])  - 1])))**2
    except:
        RMSE_user_item += (int(rating['rating']) - int(R_global))**2
    
    try:
        RMSE_user_item_gamma += (int(rating['rating']) - int((alpha*R_user[int(rating['uid'])  - 1] + beta*R_item[int(rating['mid'])  - 1]) + gamma))**2
    except:
        RMSE_user_item_gamma += (int(rating['rating']) - int(R_global))**2
            
RMSE_global = np.sqrt(RMSE_global/len(Rating_test))
RMSE_item = np.sqrt(RMSE_item/len(Rating_test))
RMSE_user = np.sqrt(RMSE_user/len(Rating_test))
RMSE_user_item = np.sqrt(RMSE_user_item/len(Rating_test))
RMSE_user_item_gamma = np.sqrt(RMSE_user_item_gamma/len(Rating_test))

Berekenen van de resultaten van de training, de MAE!

#Dit klopt buiten het feit dat alpha en beta nog niet op orde zijn.

In [745]:
#Mean absolute error
MAE_global = 0
MAE_item = 0
MAE_user = 0
MAE_user_item = 0
MAE_user_item_gamma = 0

for rating in Rating_test:
    MAE_global += abs(int(rating['rating']) - int(R_global))   
    
    try:
        MAE_item += abs(int(rating['rating']) - R_item[int(rating['mid']) - 1])
    except:
        MAE_item += abs(int(rating['rating']) - int(R_global))
        
    try:
        MAE_user += abs(int(rating['rating']) - R_user[int(rating['uid']) - 1])
    except:
        MAE_user += abs(int(rating['rating']) - int(R_global))
    
    try:
        MAE_user_item += abs(int(rating['rating']) - int((alpha*R_user[int(rating['uid']) - 1] + beta*R_item[str(rating['mid']) - 1])))
    except:
        MAE_user_item += abs(int(rating['rating']) - int(R_global))
    
    try:
        MAE_user_item_gamma += abs(int(rating['rating']) - int((alpha*R_user[int(rating['uid']) - 1] + beta*R_item[int(rating['mid']) - 1]) + gamma))
    except:
        MAE_user_item_gamma += abs(int(rating['rating']) - int(R_global))

MAE_global = MAE_global/len(Rating_test)
MAE_item = MAE_item/len(Rating_test)
MAE_user = MAE_user/len(Rating_test)
MAE_user_item = MAE_user_item/len(Rating_test)
MAE_user_item_gamma = MAE_user_item_gamma/len(Rating_test)

Printen van de resultaten

In [746]:
print(RMSE_global)
print(RMSE_item)
print(RMSE_user)
print(RMSE_user_item)
print(RMSE_user_item_gamma)
print("\n")
print(MAE_global)
print(MAE_item)
print(MAE_user)
print(MAE_user_item)
print(MAE_user_item_gamma)

1.2621491455854978
0.9823658303441268
1.039434029917006
3.753347386069386
1.2621491455854978


1.0239899621079573
0.7851682906441758
0.8329949719978602
1.0239899621079573
1.0239899621079573


# Matrix Factorization section

Een standaard predictfunctie die gebruik maakt van de R_item en de R_user

In [753]:
#Predict de data aan de hand van gemiddelde per user en gemiddelde per film
def predictRating(movie, user):
    return (R_item[int(movie)] + R_user[int(user)])/2

De trainingsfunctie die mbv een errorrate een learningrate en een K. Deze trainingsfunctie beinvloed de userValue en de movieValue die essentieel zijn om de nieuwe ratings te voorspellen. 

In [754]:
'''''''''
Where:
 *  real *userValue = userFeature[featureBeingTrained];
 *  real *movieValue = movieFeature[featureBeingTrained];
 *  real  lrate = 0.001;
'''''''''
#Train de data mbv een errorrate en de standaard predictRating functie
def train(user, movie, rating):
    lrate = 0.001
    K = 0.02
    
    err = lrate * (rating - predictRating(movie, user))
    
    uv = userValue[user];    
    userValue[user] += lrate * (err * movieValue[movie] - K * userValue[user]);
    movieValue[movie] += lrate * (err * uv - K * movieValue[movie]);


De functies die het trainen en voorspellen mogelijk maken

In [814]:
#ratingsMatrix[user][movie] = sum (userFeature[f][user] * movieFeature[f][movie]) for f from 1 to 40

#verticaal user_data en horizontaal movie_data
ratingsMatrix = np.full((int(user_data[-1]['uid']), int(movie_data[-1]['mid'])), 0.01)

#initieer de userValue en de movieValue array
userValue = np.full((int(user_data[-1]['uid']), ), R_global)
movieValue = np.full((int(movie_data[-1]['mid']), ), R_global)

#Train de data op de trainingsset
for rating in Rating_train: 
    train(int(rating['uid']) - 1, int(rating['mid']) - 1, int(rating['rating']))

#Vul de rest van de waardes in op basis van de trainingsset
for user in user_data:
    for movie in movie_data:
        ratingsMatrix[int(user['uid']) - 1, int(movie['mid']) - 1] = (userValue[int(user['uid']) - 1] + movieValue[int(movie['mid']) - 1])/2

#Print het eindresultaat
print(ratingsMatrix)

[[3.46699395 3.53900487 3.54999447 ... 3.57550261 3.57622573 3.55666275]
 [3.46211469 3.53412561 3.54511522 ... 3.57062336 3.57134648 3.55178349]
 [3.46698427 3.53899519 3.5499848  ... 3.57549294 3.57621606 3.55665307]
 ...
 [3.46876521 3.54077613 3.55176574 ... 3.57727388 3.577997   3.55843401]
 [3.46232109 3.53433201 3.54532162 ... 3.57082976 3.57155288 3.55198989]
 [3.45032969 3.52234061 3.53333021 ... 3.55883835 3.55956147 3.53999849]]


In [815]:
print(RSME_two_arrays(ratingsMatrix, testing_array))
print(MAE_two_arrays(ratingsMatrix, testing_array))

1.1291770882634757
0.9517791729018711


# UV Matrix Decomposition

In [623]:
def optimize_U_cell(r, s, dim, U_matrix, V_matrix, M_matrix):
    x_above = 0
    
    for j in range(M_matrix.shape[1]):
        M_val = M_matrix[r,j]
        
        if np.isnan(M_val):
            continue
            
        
        U_el_V_el_sum = 0
        
        for k in range(dim):
            if k == s:
                continue
            U_el_V_el_sum += U_matrix[r,k] * V_matrix[k,j]
            
        x_above += V_matrix[s,j] * (M_val - U_el_V_el_sum)
        
    x_under = 0
    
    for j in range(M_matrix.shape[1]):
        if np.isnan(M_matrix[r,j]):
            continue
            
        x_under += V_matrix[s, j] ** 2
    
    return x_above / x_under if x_under != 0. else y_above / 1e10

def optimize_V_cell(r, s, dim, U_matrix, V_matrix, M_matrix):
    y_above = 0
    
    for i in range(M_matrix.shape[0]):
        M_val = M_matrix[i,s]
        
        if np.isnan(M_val):
            continue
            
        
        U_el_V_el_sum = 0
        
        for k in range(dim):
            if k == r:
                continue
            U_el_V_el_sum += U_matrix[i,k] * V_matrix[k,s]
            
        y_above += U_matrix[i,r] * (M_val - U_el_V_el_sum)
        
    y_under = 0
    
    for i in range(M_matrix.shape[1]):
        if np.isnan(M_matrix[i,s]):
            continue
            
        y_under += U_matrix[i, r] ** 2
    
    return y_above / y_under if y_under != 0. else y_above / 1e10

In [624]:
def run_opti_round_robin(U_d_matrix, V_d_matrix, M_matrix, dimensions):
    r_U, c_U = 0, 0
    r_V, c_V = 0, 0
    
    run = True
    
    while run:
        run = False
        if r_U < U_d_matrix.shape[0] and c_U < U_d_matrix.shape[1]:
            U_d_matrix[r_U, c_U] = optimize_U_cell(r_U, c_U, dimensions, U_d_matrix, V_d_matrix, M_matrix)
            run = True
            
            c_U += 1

            if c_U >= U_d_matrix.shape[1]:
                r_U += 1
                c_U = 0
            
        if r_V < V_d_matrix.shape[0] and c_V < V_d_matrix.shape[1]:
            V_d_matrix[r_V, c_V] = optimize_V_cell(r_V, c_V, dimensions, U_d_matrix, V_d_matrix, M_matrix)
            run = True
            
            c_V += 1

            if c_U >= V_d_matrix.shape[1]:
                r_V += 1
                c_V = 0
        

In [625]:
def optimize_UV_array(U_d_matrix, V_d_matrix, M_array, dimensions, reference_RSME, max_rounds=1000, improvement_threshold=1.0e-10):
    difference = None

    for i in range(1000):
        print(f"opti-cycle {i}")
        run_opti_round_robin(U_d_matrix, V_d_matrix, M_array, dimensions)

        result = RSME_two_arrays(M_array, np.matmul(U_d_matrix, V_d_matrix))
        
        improvement = reference_RSME - result if difference is None else difference - result
        print(f"RSME result round {i}: {result}, this constitutes an improvement of: {reference_RSME - result} and a difference of {improvement}")
        
        if improvement < improvement_threshold:
            print("Improvement threshold reached, optimization stopping")
            return
        
        difference = result

## Movie loading

In [820]:
import numpy as np

# users_movies_array = np.zeros((int(user_data[-1]['uid']), int(movie_data[-1]['mid'])))

# for rating in rating_data:
#     rv = list(rating.values())
#     users_movies_array[int(rv[0]) - 1, int(rv[1]) - 1] = float(rv[2])
    
# users_movies_array = np.where(users_movies_array == 0., np.NaN, users_movies_array)

users_movies_array_no_nans = replace_nans(training_array)

users_movies_array_pp, row_mean, column_mean = preprocess_array(users_movies_array)

<ipython-input-794-c1215def58d5>:9: RuntimeWarning: Mean of empty slice
  copy_arr[inds] = np.take(np.nanmean(copy_arr, axis=0), inds[1])


In [825]:
mean_users_movies_array = np.nanmean(users_movies_array_pp)
dimensions = 5

start_value = np.sqrt(mean_users_movies_array/dimensions)

users_d_matrix = np.full((users_movies_array.shape[0], dimensions), start_value)
movies_d_matrix = np.full((dimensions, users_movies_array.shape[1]), start_value)

In [826]:
base_line = RSME_two_arrays(users_movies_array, unprocess_array(np.matmul(users_d_matrix, movies_d_matrix), row_mean, column_mean))
print(f"base_line_opti: {base_line}")

optimize_UV_array(users_d_matrix, movies_d_matrix, users_movies_array_pp, dimensions, base_line)

prediction_UV_array = unprocess_array(np.matmul(users_d_matrix, movies_d_matrix), row_mean, column_mean)

print(f"Final result: {RSME_two_arrays(testing_array, prediction_UV_array)}")

base_line_opti: 1.028448620929304
opti-cycle 0
RSME result round 0: 0.8975893580392296, this constitutes an improvement of: 0.13085926289007443 and a difference of 0.13085926289007443
opti-cycle 1
RSME result round 1: 0.8977229743863595, this constitutes an improvement of: 0.1307256465429445 and a difference of -0.00013361634712993897
Improvement threshold reached, optimization stopping
Final result: 1.043038206984398


In [827]:
print(RSME_two_arrays(prediction_UV_array, testing_array))
print(MAE_two_arrays(prediction_UV_array, testing_array))

1.043038206984398
0.839151553056242
